In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import pickle

from datetime import datetime, date, time
from textblob import TextBlob
from textblob import Word
from pre_processing import *
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Andres
[nltk_data]     Montero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stop = stopwords.words('english')
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


In [3]:
DATA_FOLDER = './data'

tweets_col_names=['text']

tweets_dtypes = {'text': str }

cleaned_pos = False
cleaned_neg = False
cleaned_test = False
cleaned_neg_full = False
cleaned_pos_full = False



In [4]:
#verify  flag for data already cleaned
if not cleaned_neg :
    #read data from .txt to preprocess
    tweets_neg = pd.read_fwf(DATA_FOLDER + '/train_neg.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    
    #increase index to start at 1
    tweets_neg.index = tweets_neg.index +1
    #remove duplicates
    tweets_neg.drop_duplicates(inplace=True)
    test = tweets_neg.head(50)
    #apply pre_process funtioncs
    #remove stop words
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # interpret emoji
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: interpret_emoji(x))
    #separate number with letters  1234test123 =>  test 
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: split_number_text(x))
    #remove hashtag #
    tweets_neg['text'] = tweets_neg['text'].str.replace('#', '')
    #replace ur by your
    tweets_neg['text'] = tweets_neg['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: expand_contractions(x))
        #remove punctuaction
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_punctuation(x))
 
    # remove words user, url, number
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaappy => happy
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_neg = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[:10]
    freq_max_neg = list(freq_max_neg.index)
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_neg))
    
#     remove least frequent words
    freq_min_neg = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[-10:]
    freq_min_neg = list(freq_min_neg.index)
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_neg))

    #correct words with textblob
    # tweets_neg['text'] = tweets_neg['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_neg
    
    #lemmatize words
    tweets_neg['text_lema'] = tweets_neg['text'].apply(lemmatize_text)
    tweets_neg['text_lema'] = tweets_neg['text_lema'].apply(lambda x: ' '.join(x))
    tweets_neg['text_lema'] = tweets_neg['text_lema'].str.replace(',','')
    
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_neg_wstp.pickle","wb")
    pickle.dump(tweets_neg['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/tweets_neg_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x), tweets_neg['text_lema'])))
    f.close()
else:
    print("Already Trained!")
    
pickle_in = open("data/pre_processed/tweets_neg_wstp.pickle","rb")
tweets_neg_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_neg_pickle.head()
    

Saving file with preprocessed Tweets
Saved!
Opening pickle


1    vinco tresorpack difficulty object disassemble...
2         glad dot have taks tomorrow thankful startho
3    v celtic regular season were fucked if we play...
4          could actually kill that girl i am so sorry
5                find that very hard believe im afraid
Name: text_lema, dtype: object

In [5]:
if not cleaned_pos:
    #read data .txt file
    tweets_pos = pd.read_fwf(DATA_FOLDER + '/train_pos.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    # index start at 1
    tweets_pos.index = tweets_pos.index + 1
    #remove duplicates
    tweets_pos.drop_duplicates(inplace=True)
    #apply pre_process funtioncs
    #remove stop words
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # interpret emoji
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: interpret_emoji(x))
    #separate number with letters  1234test123 =>  test 
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: split_number_text(x))
    print(tweets_pos['text'].count())
    #remove hashtag #
    tweets_pos['text'] = tweets_pos['text'].str.replace('#', '')
    #replace ur by your
    tweets_pos['text'] = tweets_pos['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: expand_contractions(x))
        #remove punctuaction
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_punctuation(x))
 
    # remove words user, url, number
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaappy => happy
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[:10]
    freq_max_pos = list(freq_max_pos.index)
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_pos))
    
    #remove least frequent words
    freq_min_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[-10:]
    freq_min_pos = list(freq_min_pos.index)
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_pos))

    #correct words with textblob
    # tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_pos
    
    #lemmatize words
    tweets_pos['text_lema'] = tweets_pos['text'].apply(lemmatize_text)
    tweets_pos['text_lema'] = tweets_pos['text_lema'].apply(lambda x: ' '.join(x))
    tweets_pos['text_lema'] = tweets_pos['text_lema'].str.replace(',','')
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_pos_wstp.pickle","wb")
    pickle.dump(tweets_pos['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/tweets_pos_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x), tweets_pos['text_lema'])))
    f.close()
else:
    print("Already Trained!")
pickle_in = open("data/pre_processed/tweets_pos_wstp.pickle","rb")
tweets_pos_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_pos_pickle.head()

90233
Saving file with preprocessed Tweets
Saved!
Opening pickle


1    dunno justin read mention or only justin god k...
2    because yoyour logic so dumb will even crop ou...
3      just put casper in box looved battle crakkbitch
4    thanks sir do trip lil mama just keep doin ya ...
5    visiting brother tmr bestest birthday gift eveerr
Name: text_lema, dtype: object

In [6]:
tweets_neg_pickle.head(20)

1     vinco tresorpack difficulty object disassemble...
2          glad dot have taks tomorrow thankful startho
3     v celtic regular season were fucked if we play...
4           could actually kill that girl i am so sorry
5                 find that very hard believe im afraid
6                   wish could be out all night tonight
7                                    got kicked out wgm
8                rt yes she tell it lip are closed okay
9                                    why she so perfect
10    hi harry did havea good time au didnt get see ...
11    introduction programming with + + nd edition t...
14                                        i am white aw
15             dan love miss do be sad wheresthegeneral
16         so many wonderful building dc but still miss
17         it annoying because secretly find it so good
18    post boom spanish american fiction suny series...
19    layer heart paperback this joyourney wa inspir...
21                guess who texted me again want

In [7]:
tweets_pos_pickle.head(20)

1     dunno justin read mention or only justin god k...
2     because yoyour logic so dumb will even crop ou...
3       just put casper in box looved battle crakkbitch
4     thanks sir do trip lil mama just keep doin ya ...
5     visiting brother tmr bestest birthday gift eveerr
6      yay lifecompleted tweet facebook let know please
7     dnextalbumtitle feel rollercoaster of life son...
8     workin hard or hardly workin rt at hardee's wi...
9                         saw i will be replying in bit
10                                     this were belong
11                                  andd cheer national
12    we send an invitation shop on line here will f...
13                       just woke up finna go chyourch
14                             agreed more day left tho
15                                  monet with katemelo
16    like damm lexis got lot say when your on twitt...
17    grateful today dream fulfilled heart so full f...
18                        at home affair shall d

In [ ]:
#verify  flag for data already cleaned
if not cleaned_neg_full :
    #read data from .txt to preprocess
    tweets_neg_full = pd.read_fwf(DATA_FOLDER + '/train_neg_full.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    
    #increase index to start at 1
    tweets_neg_full.index = tweets_neg_full.index +1
    #remove duplicates
    tweets_neg_full.drop_duplicates(inplace=True)
    test = tweets_neg_full.head(50)
    #apply pre_process funtioncs
    #remove stop words
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    print("emoji")
    # interpret emoji
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: interpret_emoji(x))
    print("split numbers")
    #separate number with letters  1234test123 =>  test 
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: split_number_text(x))
    print("hastag")
    #remove hashtag #
    tweets_neg_full['text'] = tweets_neg_full['text'].str.replace('#', '')
    print("ur your")
    #replace ur by your
    tweets_neg_full['text'] = tweets_neg_full['text'].str.replace('ur', 'your')
    print("expand contractions")
    #expand contractions don't => do not
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: expand_contractions(x))
    print("punctuaction")
        #remove punctuaction
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: remove_punctuation(x))
    print("remove words")
    # remove words user, url, number
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: remove_words(x))
    print("haaapy to haapy")
    # replace more letters haaaaaaaaaappy => happy
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: replace_moreletters(x))
    
    print("frequent words")
    #remove most frequent words 
    freq_max_neg_full = pd.Series(' '.join(tweets_neg_full['text']).split()).value_counts()[:10]
    freq_max_neg_full = list(freq_max_neg_full.index)
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_neg_full))
    
    print("least frequent words")
    #remove least frequent words
    freq_min_neg_full = pd.Series(' '.join(tweets_neg_full['text']).split()).value_counts()[-10:]
    freq_min_neg_full = list(freq_min_neg_full.index)
    tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_neg_full))

    #correct words with textblob
    # tweets_neg['text'] = tweets_neg['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_neg
    print("lemmatizing")
    #lemmatize words
    tweets_neg_full['text_lema'] = tweets_neg_full['text'].apply(lemmatize_text)
    tweets_neg_full['text_lema'] = tweets_neg_full['text_lema'].apply(lambda x: ' '.join(x))
    tweets_neg_full['text_lema'] = tweets_neg_full['text_lema'].str.replace(',','')
    
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_neg_full_wstp.pickle","wb")
    pickle.dump(tweets_neg_full['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/tweets_neg_full_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x), tweets_neg_full['text_lema'])))
    f.close()
else:
    print("Already Trained!")
    
pickle_in = open("data/pre_processed/tweets_neg_full_wstp.pickle","rb")
tweets_neg_full_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_neg_full_pickle.head()

emoji


In [ ]:
if not cleaned_pos_full:
    #read data .txt file
    tweets_pos_full = pd.read_fwf(DATA_FOLDER + '/train_pos_full.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    # index start at 1
    tweets_pos_full.index = tweets_pos_full.index + 1
    #remove duplicates
    tweets_pos_full.drop_duplicates(inplace=True)
    #apply pre_process funtioncs
    #remove stop words
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    print("Interpreting emojis!!")
     # interpret emoji
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: interpret_emoji(x))
    print("Separating Numbers!!")
    #separate number with letters  1234test123 =>  test 
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: split_number_text(x))
#     print(tweets_pos_full['text'].count())
    print("Remove Hashtags!!")
    #remove hashtag #
    tweets_pos_full['text'] = tweets_pos_full['text'].str.replace('#', '')
    print("Replace ur by your!!")
    #replace ur by your
    tweets_pos_full['text'] = tweets_pos_full['text'].str.replace('ur', 'your')
    print("expand contractions!!")
    #expand contractions don't => do not
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: expand_contractions(x))
    #remove punctuaction
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: remove_punctuation(x))
    print("remove words!!")
    # remove words user, url, number
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: remove_words(x))
    print("Replace more leeters!!")
    # replace more letters haaaaaaaaaappy => happy
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: replace_moreletters(x))
    
    print("Remove most frequent words!!")
    #remove most frequent words 
    freq_max_pos_full = pd.Series(' '.join(tweets_pos_full['text']).split()).value_counts()[:10]
    freq_max_pos_full = list(freq_max_pos_full.index)
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_pos_full))
    
    print("Remove least frequent words!!")
    #remove least frequent words
    freq_min_pos_full = pd.Series(' '.join(tweets_pos_full['text']).split()).value_counts()[-10:]
    freq_min_pos_full = list(freq_min_pos_full.index)
    tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_pos_full))

    #correct words with textblob
    # tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_pos
    
    print("Lemmatizing!!")
    #lemmatize words
    tweets_pos_full['text_lema'] = tweets_pos_full['text'].apply(lemmatize_text)
    print("joining after lematize")
    tweets_pos_full['text_lema'] = tweets_pos_full['text_lema'].apply(lambda x: ' '.join(x))
    print("removing commas ")
    tweets_pos_full['text_lema'] = tweets_pos_full['text_lema'].str.replace(',','')
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_pos_full_wstp.pickle","wb")
    pickle.dump(tweets_pos_full['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/tweets_pos_full_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x), tweets_pos_full['text_lema'])))
    f.close()
else:
    print("Already Trained!")
pickle_in = open("data/pre_processed/tweets_pos_full_wstp.pickle","rb")
tweets_pos_full_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_pos_full_pickle.head()

In [ ]:
if not cleaned_test:  
    #read data from .txt
    test_data = pd.read_fwf(DATA_FOLDER + '/test_data.txt', header=None)
    #index start at 1
    test_data.index = test_data.index +1
    #remove duplicates
    test_data.drop_duplicates(inplace=True)
    test_data.drop(columns=[1,2], inplace=True)
    test_data.rename(columns={0:'text'}, inplace= True)
    
    #apply pre_process funtioncs
    #remove stop words
#     test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # interpret emoji
    test_data['text'] = test_data['text'].apply(lambda x: interpret_emoji(x))
    #separate number with letters  1234test123 => test 
    test_data['text'] = test_data['text'].apply(lambda x: split_number_text(x))
    #remove hashtag #
    test_data['text'] = test_data['text'].str.replace('#', '')
    #replace ur by your
    test_data['text'] = test_data['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    test_data['text'] = test_data['text'].apply(lambda x: expand_contractions(x))
        #remove punctuaction
    test_data['text'] = test_data['text'].apply(lambda x: remove_punctuation(x))
 
    # remove words user, url, number
    test_data['text'] = test_data['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaapy => hapy
    test_data['text'] = test_data['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words from poss_full
    freq_max_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[:10]
    freq_max_test = list(freq_max_test.index)
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_pos_full))
    
    #remove most frequent words from neg_full
    freq_max_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[:10]
    freq_max_test = list(freq_max_test.index)
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_neg_full))
    
    
    #remove least frequent words from poss_full
    freq_min_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[-10:]
    freq_min_test = list(freq_min_test.index)
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_pos_full))
    
     #remove least frequent words from poss_full
    freq_min_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[-10:]
    freq_min_test = list(freq_min_test.index)
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_neg_full))

    #correct words with textblob
    # tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_pos
    
    #lemmatize words
    test_data['text_lema'] = test_data['text'].apply(lemmatize_text)
    test_data['text_lema'] = test_data['text_lema'].apply(lambda x: ' '.join(x))
    test_data['text_lema'] = test_data['text_lema'].str.replace(',','')
    index = test_data.index.to_series()
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/test_data_wstp.pickle","wb")
    pickle.dump(test_data['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
    f = open("data/pre_processed/test_data_wstp.txt", "w")
    f.write("\n".join(map(lambda x: str(x[0]+1) + ',' + str(x[1]), enumerate(test_data['text_lema']))))
    f.close()
else:
    print("Already Trained!")
pickle_in = open("data/pre_processed/test_data_wstp.pickle","rb")
test_data_pickle = pickle.load(pickle_in)
print("Opening pickle")
test_data_pickle.head()

In [ ]:
test_data_pickle.head(20)

In [ ]:
test_data_pickle